In [18]:
import platform
import torch

def showinfo(tip, info):
    print("{}:{}".format(tip,info))

showinfo("操作系统及版本信息",platform.platform())
showinfo('系统位数', platform.architecture())
showinfo('pytorch版本', torch.__version__)
showinfo('cuda版本', torch.version.cuda)
showinfo('cudnn版本', torch.backends.cudnn.version())

操作系统及版本信息:Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
系统位数:('64bit', 'ELF')
pytorch版本:2.3.0
cuda版本:12.1
cudnn版本:8902


## 根据理论基础note中的导数简单实现一个神经网络

In [19]:
import random
import math


# 激活函数及其导数
def sigmoid(x):
    return 1 / (1 + math.exp(-x))


def sigmoid_derivative(x):
    return x * (1 - x)


class Node:
    def __init__(self, weight=None, bias=None, activate_func=sigmoid, activate_func_derivative=sigmoid_derivative):
        self.weight = weight if weight is not None else random.uniform(-1, 1)
        self.bias = bias if bias is not None else random.uniform(-1, 1)
        self.activate_func = activate_func
        self.activate_func_derivative = activate_func_derivative
        self.output = None
        self.inputs = None
        self.delta = None

    # 前向传播：激活函数（上一层输出 * 这一层weight + bias）
    def forward(self, inputs):
        self.inputs = inputs
        self.output = self.activate_func(sum(inputs[i] * self.weight[i] for i in range(len(inputs))) + self.bias)
        return self.output

    # 反向传播：差值 = 节点激活函数的梯度
    def backward(self, y, learning_rate):
        self.delta = y * self.activate_func_derivative(self.output)
        for i in range(len(self.weight)):
            self.weight[i] += learning_rate * self.delta * self.inputs[i]
        self.bias += learning_rate * self.delta
        return self.delta


In [20]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_layer = [Node(weight=[random.uniform(-1, 1) for _ in range(input_size)]) for _ in range(hidden_size)]
        self.output_layer = [Node(weight=[random.uniform(-1, 1) for _ in range(hidden_size)], activate_func=lambda x: x,
                                  activate_func_derivative=lambda x: 1) for _ in range(output_size)]

    def forward(self, x):
        hidden_outputs = [node.forward(x) for node in self.hidden_layer]
        final_outputs = [node.forward(hidden_outputs) for node in self.output_layer]
        return final_outputs

    def backward(self, y, learning_rate):
        output_errors = [yi - yi_pred.output for yi, yi_pred in zip(y, self.output_layer)]
        hidden_errors = [node.backward(error, learning_rate) for node, error in zip(self.output_layer, output_errors)]
        for node in self.hidden_layer:
            node.backward(sum(hidden_errors), learning_rate)

    def train(self, x, y, epochs, learning_rate):
        for epoch in range(epochs):
            for xi, yi in zip(x, y):
                self.forward([xi])
                self.backward([yi], learning_rate)
            if epoch % 1000 == 0:
                predictions = [self.forward([xi])[0] for xi in x]
                loss = sum((yi - yi_pred) ** 2 for yi, yi_pred in zip(y, predictions)) / len(y)
                print(f'Epoch {epoch}, Loss: {loss}')

In [21]:
# 初始化数据
x = [-2, -1, 0, 1, 2, 3]
y = [4, 1, 0, 1, 4, 9]  # 一元二次次函数问题： y=x^2

# 初始化和训练神经网络
input_size = 1
hidden_size = 20  # 隐藏层节点数
output_size = 1
epochs = 10000
learning_rate = 0.01  # 使用较小的学习率

nn = NeuralNetwork(input_size, hidden_size, output_size)
nn.train(x, y, epochs, learning_rate)

# 测试训练后的神经网络
predictions = [nn.forward([xi])[0] for xi in x]
print("Final predictions:")
print(predictions)

Epoch 0, Loss: 9.910500579219743
Epoch 1000, Loss: 0.06519984016543566
Epoch 2000, Loss: 0.023668808720770512
Epoch 3000, Loss: 0.012736884810698958
Epoch 4000, Loss: 0.007905091943442164
Epoch 5000, Loss: 0.005282467051771467
Epoch 6000, Loss: 0.0036927461840573525
Epoch 7000, Loss: 0.0026608095287602523
Epoch 8000, Loss: 0.0019592242322174452
Epoch 9000, Loss: 0.0014661825116508958
Final predictions:
[3.9830770013466616, 1.0412549356682734, -0.034841887954276785, 0.9810572026551035, 4.051825554052546, 8.979481589664744]


## NN by numpy

In [22]:
import numpy as np

class Node:
    def __init__(self, input_size, activate_func=sigmoid, activate_func_derivative=sigmoid_derivative):
        ## numpy初始化向量
        self.weights = np.random.uniform(-1, 1, input_size)
        self.bias = np.random.uniform(-1, 1)
        self.activate_func = activate_func
        self.activate_func_derivative = activate_func_derivative
        self.output = None
        self.inputs = None
        self.delta = None

    def forward(self, inputs):
        self.inputs = np.array(inputs)
        self.output = self.activate_func(np.dot(self.inputs, self.weights) + self.bias) ## 可以通过numpy的矩阵运算加速
        return self.output

    def backward(self, y, learning_rate):
        self.delta = y * self.activate_func_derivative(self.output)
        self.weights += learning_rate * self.delta * self.inputs
        self.bias += learning_rate * self.delta
        return self.delta


class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_layer = [Node(input_size) for _ in range(hidden_size)]
        self.output_layer = [Node(hidden_size, activate_func=lambda x: x, activate_func_derivative=lambda x: 1) for _ in range(output_size)]

    def forward(self, x):
        hidden_outputs = np.array([node.forward(x) for node in self.hidden_layer])
        final_outputs = np.array([node.forward(hidden_outputs) for node in self.output_layer])
        return final_outputs

    def backward(self, y, learning_rate):
        output_errors = np.array([yi - yi_pred.output for yi, yi_pred in zip(y, self.output_layer)])
        hidden_errors = np.array([node.backward(error, learning_rate) for node, error in zip(self.output_layer, output_errors)])
        for node in self.hidden_layer:
            node.backward(np.sum(hidden_errors), learning_rate)

    def train(self, x, y, epochs, learning_rate):
        for epoch in range(epochs):
            for xi, yi in zip(x, y):
                self.forward([xi])
                self.backward([yi], learning_rate)
            if epoch % 1000 == 0:
                predictions = [self.forward([xi])[0] for xi in x]
                loss = np.mean((np.array(y) - np.array(predictions)) ** 2)
                print(f'Epoch {epoch}, Loss: {loss}')

In [23]:
# 初始化数据
x = [-2, -1, 0, 1, 2, 3]
y = [4, 1, 0, 1, 4, 9]  # 一元二次次函数问题： y=x^2

# 初始化和训练神经网络
input_size = 1
hidden_size = 20  # 隐藏层节点数
output_size = 1
epochs = 10000
learning_rate = 0.01  # 使用较小的学习率

nn = NeuralNetwork(input_size, hidden_size, output_size)
nn.train(x, y, epochs, learning_rate)

# 测试训练后的神经网络
predictions = [nn.forward([xi])[0] for xi in x]
print("Final predictions:")
print(predictions)

Epoch 0, Loss: 12.690397553342853
Epoch 1000, Loss: 0.08446104376154805
Epoch 2000, Loss: 0.034334094827737714
Epoch 3000, Loss: 0.020305656419910563
Epoch 4000, Loss: 0.013638394540434957
Epoch 5000, Loss: 0.009780126692921802
Epoch 6000, Loss: 0.007301030743069145
Epoch 7000, Loss: 0.00560291273924175
Epoch 8000, Loss: 0.0043889982859892105
Epoch 9000, Loss: 0.0034940729742537874
Final predictions:
[3.971084067792658, 1.0710872830900582, -0.025178577447386052, 0.9359522178782205, 4.074898247479153, 8.973928099986354]


## NN by pytorch

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim


class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        ## 一个隐藏层：全连接 + sigmoid激活函数
        self.hidden_layer = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Sigmoid(),
        )
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.hidden_layer(x)
        x = self.output_layer(x)
        return x


In [25]:
# 初始化数据
x = [-2, -1, 0, 1, 2, 3]
y = [4, 1, 0, 1, 4, 9]  # 一元二次次函数问题： y=x^2# 数据标准化

# 转换为Tensor
x = torch.tensor(x, dtype=torch.float32).unsqueeze(1)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
print('x: ', x)
print('y: ', y)

# 初始化和训练神经网络
input_size = 1
hidden_size = 10
output_size = 1
epochs = 10000
learning_rate = 0.001

model = NeuralNetwork(input_size, hidden_size, output_size)
# 使用均方误差作为损失函数，最速梯度下降作为优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# 测试训练后的神经网络
model.eval()
with torch.no_grad():
    predictions = model(x)
    print("Final predictions:")
    print(predictions)

x:  tensor([[-2.],
        [-1.],
        [ 0.],
        [ 1.],
        [ 2.],
        [ 3.]])
y:  tensor([[4.],
        [1.],
        [0.],
        [1.],
        [4.],
        [9.]])
Epoch 0, Loss: 18.863506317138672
Epoch 1000, Loss: 6.528347492218018
Epoch 2000, Loss: 4.087300777435303
Epoch 3000, Loss: 2.0222127437591553
Epoch 4000, Loss: 0.466418594121933
Epoch 5000, Loss: 0.07475780695676804
Epoch 6000, Loss: 0.017330272123217583
Epoch 7000, Loss: 0.004752637818455696
Epoch 8000, Loss: 0.0021106048952788115
Epoch 9000, Loss: 0.0011547842295840383
Final predictions:
tensor([[3.9986],
        [0.9991],
        [0.0300],
        [0.9596],
        [4.0206],
        [8.9923]])


## 补充：n元n次方程模拟，二元一次为例

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim


class NormalNN(nn.Module):

    def __init__(self, in_dimension, out_dimension):
        super(NormalNN, self).__init__()

        self.hide_layer_func = nn.Linear(in_dimension, 10)
        self.out_layer_func = nn.Linear(10, out_dimension)
        self.activate_func = nn.ReLU()

    def forward(self, x):
        x = self.hide_layer_func(x)
        x = self.activate_func(x)
        x = self.out_layer_func(x)
        return x

In [27]:
# 创建模型实例
model = NormalNN(2, 1)

# 使用均方误差作为损失函数，最速梯度下降作为优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)  # 学习率为0.01

# 输入数据
inputs = torch.tensor([[1.0, 2.0], [2.0, 3.0], [3.0, 4.0]])
outputs = torch.tensor([[3.0], [5.0], [7.0]])

num_epochs = 500
for epoch in range(num_epochs):
    # 前向传播
    predictions = model(inputs)
    # 计算损失
    loss = criterion(predictions, outputs)
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# 测试模型
test_input = torch.tensor([[4.0, 5.0], [3.14, 9.29]])
with torch.no_grad():
    prediction = model(test_input)
    print("Prediction:", prediction.flatten().tolist())

Epoch [10/500], Loss: 0.0365
Epoch [20/500], Loss: 0.0323
Epoch [30/500], Loss: 0.0285
Epoch [40/500], Loss: 0.0251
Epoch [50/500], Loss: 0.0221
Epoch [60/500], Loss: 0.0194
Epoch [70/500], Loss: 0.0171
Epoch [80/500], Loss: 0.0150
Epoch [90/500], Loss: 0.0131
Epoch [100/500], Loss: 0.0115
Epoch [110/500], Loss: 0.0100
Epoch [120/500], Loss: 0.0088
Epoch [130/500], Loss: 0.0076
Epoch [140/500], Loss: 0.0066
Epoch [150/500], Loss: 0.0058
Epoch [160/500], Loss: 0.0050
Epoch [170/500], Loss: 0.0044
Epoch [180/500], Loss: 0.0038
Epoch [190/500], Loss: 0.0033
Epoch [200/500], Loss: 0.0028
Epoch [210/500], Loss: 0.0025
Epoch [220/500], Loss: 0.0021
Epoch [230/500], Loss: 0.0018
Epoch [240/500], Loss: 0.0016
Epoch [250/500], Loss: 0.0014
Epoch [260/500], Loss: 0.0012
Epoch [270/500], Loss: 0.0010
Epoch [280/500], Loss: 0.0009
Epoch [290/500], Loss: 0.0008
Epoch [300/500], Loss: 0.0006
Epoch [310/500], Loss: 0.0006
Epoch [320/500], Loss: 0.0005
Epoch [330/500], Loss: 0.0004
Epoch [340/500], Lo